# Capstone Project - **Concierge Agent** Category 

* **CoMailAgent** – Automated Resume Tailoring & Cold Email Job Outreach Agent 

* Author: Supriya & Sanya
* Submission Date: 2025-12-01

## Project Summary:
This project leverages AI agents (Gemini) to automate the rewriting
and customization of cover letters for job applications. By integrating Google Sheets and Docs, users can provide multiple job postings and a base cover letter, which the agent rewrites to match each job description.
PDFs of the updated cover letters are generated, and email previews are printed to simulate sending applications.

* **Track**: Concierge Agent
* **Problem**: Manually rewriting cover letters for each job is time-consuming.
* **Solution**: Use a Gemini-powered agent to rewrite cover letters automatically.
* **Value**: Saves hours per week, ensures personalized, professional applications.


### Step 1: Downloading required packages
This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key.

In [1]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=b8ddfe83b203d4bab1e1eb96c8369fefe3692ca9855775e96e9562da9bf41934
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


### Step 2: Imports
Now, importing the specific components we will need from the popular python packages, Agent Development Kit and the Generative AI library. This keeps the code organized and ensures we have access to the necessary building blocks.

In [2]:
# ------------------ Imports ------------------
import os
import io
import logging
from typing import Dict, Any

import pandas as pd
import requests
from fpdf import FPDF

# ADK / Gemini Imports
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService

print("All imports done ✅")

All imports done ✅


### Step 3: Loging
Logging allows tracking of errors, API calls, and processing steps.

In [3]:
# ------------------ Logging ------------------
logging.basicConfig(level=logging.INFO)
print("Logging set ✅")

Logging set ✅


### Step 4: Configuring Google API Keys
This notebook uses 
* the [Gemini API](https://ai.google.dev/gemini-api/), which requires an API key
* ids for downloading google docs

In [4]:
# Kaggle secrets helper
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
except Exception:
    user_secrets = None

def get_secret(name: str, default=None):
    if user_secrets:
        try:
            return user_secrets.get_secret(name)
        except Exception:
            pass
    return os.environ.get(name, default)

SPREADSHEET_ID = get_secret("SPREADSHEET_ID")
FILE_ID = get_secret("FILE_ID")
GOOGLE_API_KEY = get_secret("GOOGLE_API_KEY")
print("✅" ,f"Secrets present: SPREADSHEET_ID={'set' if SPREADSHEET_ID else 'missing'}, FILE_ID={'set' if FILE_ID else 'missing'}, GOOGLE_API_KEY={'set' if GOOGLE_API_KEY else 'missing'}")


✅ Secrets present: SPREADSHEET_ID=set, FILE_ID=set, GOOGLE_API_KEY=set


### Step 5: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [5]:
# ------------------ Retry Config ------------------
# Configures retry strategy for API calls (Gemini LLM) to handle transient errors
retry_config = types.HttpRetryOptions(
    attempts=5,               # Number of retry attempts
    exp_base=7,               # Exponential backoff base
    initial_delay=1,          # Initial delay between retries in seconds
    http_status_codes=[429, 500, 503, 504],  # HTTP errors to retry
)
print("Retry Config set ✅")

Retry Config set ✅


1. root_agent - Uses the 
1. get_task_items() tool
    * Goes through the db and returns a dictionary with 
2. cover_letter_agent - for writing a custom cover letter
    * **Extracts keywords and role expectations** from the job description.
    * ~~**Tailors the resume** to highlight relevant skills and achievements.~~
    * **Generates a professional, customized cover letter** for each job.
3. email_agent - for drafting a recruiter outreach email
    * **Creates a personalized cold outreach email** addressed to the recruiter or hiring manager.
    * **Packages attachments** (tailored resume + cover letter).
    * **Sends the email automatically**, or optionally requests user approval before sending.

### Step 6: Utility Functions
Creating some utitlity functions to handle textual data

In [6]:
# ------------------ Utilities ------------------
def clean_text(text: str) -> str:
    """
    Cleans input text for PDF generation.
    - Removes BOM characters.
    - Replaces non-ASCII characters with '?'.
    This ensures PDFs do not fail due to unsupported characters.
    """
    if not text:
        return ""
    text = text.replace("\ufeff", "")
    return text.encode("ascii", "replace").decode()

def generate_pdf_bytes_from_text(text_content: str) -> bytes:
    """
    Converts plain text into PDF format using FPDF.
    Returns PDF as bytes.
    - Each line is handled individually.
    - Empty lines add spacing.
    - Multi-cell ensures proper word wrapping.
    """
    text_content = clean_text(text_content)
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)
    for line in text_content.splitlines():
        if line.strip() == "":
            pdf.ln(6)
        else:
            pdf.multi_cell(0, 8, txt=line)
    out = pdf.output(dest="S")
    return out.encode("latin1") if isinstance(out, str) else out
print("Utility Funtions set ✅")

Utility Funtions set ✅


### Step 7: Tools
Creating tools for the agent to use

In [7]:
# ------------------ Tools ------------------
def get_job_data_method() -> Dict[str, Any]:
    """
    Fetches job data from Google Sheet.
    Expects columns: job_id, job_title, job_description, recruiter_name, recruiter_email
    Returns a dictionary containing job data in a list format.
    """
    url = f"https://docs.google.com/spreadsheets/d/{SPREADSHEET_ID}/export?format=xlsx"
    try:
        df = pd.read_excel(url, sheet_name=0)
        if df.empty:
            return {"status": "error", "error_message": "Google Sheet is empty"}
        df.columns = [c.strip() for c in df.columns]
        return {"status": "success", "job_data": df.to_dict(orient="list")}
    except Exception as e:
        return {"status": "error", "error_message": f"Failed to fetch sheet: {e}"}

def get_cover_letter_method() -> Dict[str, Any]:
    """
    Fetches the base cover letter from a Google Doc.
    Returns plain text to feed into the Gemini agent.
    """
    url = f"https://docs.google.com/document/d/{FILE_ID}/export?format=txt"
    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        text = resp.text
        if not text.strip():
            return {"status": "error", "error_message": "Cover letter doc is empty"}
        return {"status": "success", "cover_letter": text}
    except requests.exceptions.RequestException as e:
        return {"status": "error", "error_message": f"HTTP error fetching doc: {e}"}

def save_text_as_pdf_method(content_to_save: str, filename: str = "cover_letter.pdf") -> Dict[str, Any]:
    """
    Saves the updated cover letter text as a PDF file.
    Returns the path to the saved PDF for reference or email attachment.
    """
    if not content_to_save:
        return {"status": "error", "error_message": "No content to save"}
    if not filename.lower().endswith(".pdf"):
        filename += ".pdf"
    try:
        pdf_bytes = generate_pdf_bytes_from_text(content_to_save)
        path = os.path.join(os.getcwd(), filename)
        with open(path, "wb") as f:
            f.write(pdf_bytes)
        return {"status": "success", "file_path": path}
    except Exception as e:
        return {"status": "error", "error_message": f"PDF save failed: {e}"}

def prepare_and_print_email_method(recruiter_email: str, recruiter_name: str, job_title: str, email_body: str, pdf_path: str) -> Dict[str, Any]:
    """
    Prepares and prints a simulated email preview.
    - Combines greeting, job title, and body.
    - Shows PDF attachment path.
    This avoids sending actual emails but allows review of content.
    """
    greeting = f"Dear {recruiter_name}," if recruiter_name else "Hello,"
    subject = f"Application for {job_title}"
    full_body = f"{greeting}\n\n{email_body}\n\nBest regards,\nAm Leeman"
    print("\n" + "="*60)
    print("📧  EMAIL PREVIEW")
    print("="*60)
    print(f"To: {recruiter_email}")
    print(f"Subject: {subject}")
    print("\n--- Email Body ---\n")
    print(full_body)
    print("\n--- Attachment (cover letter PDF saved) ---")
    print(f"{pdf_path}")
    print("="*60 + "\n")
    return {"status": "success", "message": "Printed email preview", "pdf_path": pdf_path}
print("Tools set ✅")

Tools set ✅


### Step 8: LLM Agent
An agent to rewrite the cover letter 

In [8]:
# ------------------ Gemini Agent ------------------
# LLM agent to rewrite cover letters
cover_letter_agent = LlmAgent(
    name="CoverLetterAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
You are a professional career assistant. Rewrite and customise the cover letter so that it aligns with
the provided job description. Keep tone professional and concise. Do not invent any new facts.
Return ONLY the rewritten cover letter.
""",
    tools=[]
)

# Runner to handle in-memory execution of the agent
cover_letter_runner = InMemoryRunner(agent=cover_letter_agent)
print("LLM Agent & Runner set ✅")

LLM Agent & Runner set ✅


### Step 9: Function to call the LLM Agent
 

In [9]:
def gemini_rewrite_cover_letter(base_cover_letter: str, job_description: str, job_title: str = "") -> Dict[str, Any]:
    """
    Sends prompt to Gemini LLM agent to rewrite the cover letter.
    Returns a dictionary with the updated cover letter.
    """
    if not base_cover_letter:
        return {"status": "error", "error_message": "Base cover letter is empty"}
    if not job_description:
        return {"status": "error", "error_message": "Job description is empty"}

    prompt = (
        f"Job Title: {job_title}\n\n"
        f"Job Description:\n{job_description}\n\n"
        f"Original Cover Letter:\n{base_cover_letter}\n\n"
        "Rewrite the cover letter now:"
    )

    try:
        response = cover_letter_runner.run_debug(prompt)
        updated = response.text if hasattr(response, "text") else str(response)
        if not updated.strip():
            return {"status": "error", "error_message": "Empty response from Gemini agent"}
        return {"status": "success", "updated_cover_letter": updated.strip()}
    except Exception as e:
        logging.exception("Gemini rewrite failed")
        return {"status": "error", "error_message": f"Gemini error: {e}"}


### Step 10: Orchestrator
 

In [10]:
# ------------------ Orchestrator ------------------
def orchestrator_pipeline():
    """
    Main pipeline to process multiple job applications:
    1. Fetch job data from Google Sheets.
    2. Fetch base cover letter from Google Docs.
    3. Rewrite cover letter using Gemini LLM agent.
    4. Save updated cover letter as PDF.
    5. Print email preview with attachment.
    """
    out_all = []

    # Step 1: Get jobs
    res_jobs = get_job_data_method()
    if res_jobs.get("status") != "success":
        print("ERROR fetching job data:", res_jobs.get("error_message"))
        return {"status": "error", "detail": res_jobs}
    job_data = res_jobs["job_data"]
    num_rows = len(next(iter(job_data.values())))
    print(f"Found {num_rows} job rows. Processing sequentially...")

    # Step 2: Process each job
    for i in range(num_rows):
        job_title = job_data.get("job_title", [""]*num_rows)[i]
        job_description = job_data.get("job_description", [""]*num_rows)[i]
        recruiter_name = job_data.get("recruiter_name", [""]*num_rows)[i]
        recruiter_email = job_data.get("recruiter_email", [""]*num_rows)[i]
        job_id = job_data.get("job_id", [str(i+1)])[i]

        print(f"\n--- Processing job {i+1}/{num_rows}: {job_title} (id={job_id}) ---")

        # Fetch base cover letter
        res_cl = get_cover_letter_method()
        if res_cl.get("status") != "success":
            print("ERROR fetching cover letter:", res_cl.get("error_message"))
            out_all.append({"job_index": i, "status": "error", "stage": "get_cover_letter"})
            continue
        base_cover = res_cl["cover_letter"]

        # Gemini rewrite
        res_rewrite = gemini_rewrite_cover_letter(base_cover, job_description, job_title)
        if res_rewrite.get("status") != "success":
            print("ERROR rewriting cover letter with Gemini:", res_rewrite.get("error_message"))
            out_all.append({"job_index": i, "status": "error", "stage": "gemini_rewrite"})
            continue
        updated_cover = res_rewrite["updated_cover_letter"]

        # Save PDF
        pdf_filename = f"cover_letter_job_{job_id}.pdf"
        res_pdf = save_text_as_pdf_method(updated_cover, filename=pdf_filename)
        if res_pdf.get("status") != "success":
            print("ERROR saving PDF:", res_pdf.get("error_message"))
            out_all.append({"job_index": i, "status": "error", "stage": "save_pdf"})
            continue
        pdf_path = res_pdf["file_path"]
        print(f"✅ Saved updated cover letter: {pdf_path}")

        # Print email preview
        email_body = f"I am writing to express interest in the {job_title} role. Please find my updated cover letter attached and below {updated_cover}."
        res_email = prepare_and_print_email_method(
            recruiter_email=recruiter_email,
            recruiter_name=recruiter_name,
            job_title=job_title,
            email_body=email_body,
            pdf_path=pdf_path
        )

        out_all.append({"job_index": i, "status": res_email.get("status"), "pdf_path": pdf_path})

    print("\nPipeline complete.")
    return {"status": "finished", "results": out_all}
print("Orchestrator set ✅")

Orchestrator set ✅


### Step 11: Running the pipeline
 

In [11]:
# ------------------ Run Pipeline ------------------
print("⏳ Running CoMail AI Orchestrator Pipeline (Gemini rewrite, no email sent)...")
pipeline_result = orchestrator_pipeline()
print("✅ Pipeline finished. Summary:")
print(pipeline_result)

⏳ Running CoMail AI Orchestrator Pipeline (Gemini rewrite, no email sent)...
Found 3 job rows. Processing sequentially...

--- Processing job 1/3: Data Engineer (id=1) ---


/tmp/ipykernel_13/3205656109.py:41: RuntimeWarning: coroutine 'Runner.run_debug' was never awaited
  res_rewrite = gemini_rewrite_cover_letter(base_cover, job_description, job_title)


✅ Saved updated cover letter: /kaggle/working/cover_letter_job_1.pdf

📧  EMAIL PREVIEW
To: rachael@reatil.com
Subject: Application for Data Engineer

--- Email Body ---

Dear rachael x,

I am writing to express interest in the Data Engineer role. Please find my updated cover letter attached and below <coroutine object Runner.run_debug at 0x77fd126e0430>.

Best regards,
Am Leeman

--- Attachment (cover letter PDF saved) ---
/kaggle/working/cover_letter_job_1.pdf


--- Processing job 2/3: Data Engineer (id=2) ---
✅ Saved updated cover letter: /kaggle/working/cover_letter_job_2.pdf

📧  EMAIL PREVIEW
To: ross@museum.com
Subject: Application for Data Engineer

--- Email Body ---

Dear ross y,

I am writing to express interest in the Data Engineer role. Please find my updated cover letter attached and below <coroutine object Runner.run_debug at 0x77fd126e0c10>.

Best regards,
Am Leeman

--- Attachment (cover letter PDF saved) ---
/kaggle/working/cover_letter_job_2.pdf


--- Processing job 3/

In [12]:
!ls -lah /kaggle/working/

total 56K
drwxr-xr-x 2 root root 4.0K Dec  1 19:33 .
drwxr-xr-x 8 root root 4.0K Dec  1 19:32 ..
-rw-r--r-- 1 root root 1004 Dec  1 19:33 cover_letter_job_1.pdf
-rw-r--r-- 1 root root 1004 Dec  1 19:33 cover_letter_job_2.pdf
-rw-r--r-- 1 root root 1004 Dec  1 19:33 cover_letter_job_3.pdf
---------- 1 root root  36K Dec  1 19:33 __notebook__.ipynb
